In [1]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
GPT_MODEL = 'gpt-4o-mini'
gpt_openai = OpenAI()

OLLAMA_API = "http://localhost:11434/api/chat"
OLLAMA_MODEL = "llama3.2"

'''
  define openai to use ollama at local
'''
ollama_via_openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')

API key looks good so far


In [2]:
system_message = "You are an assistant that is great at telling jokes"
user_prompt = "Tell a light-hearted joke for an audience of Data Scientists"

In [3]:
prompts = [
    {"role": "system", "content": system_message},
    {"role": "user", "content": user_prompt}
  ]

In [5]:
# GPT-4o-mini
# Temperature setting controls creativity

completion = gpt_openai.chat.completions.create(
    model=GPT_MODEL,
    messages=prompts,
    temperature=0.7
)
print(completion.choices[0].message.content)

Why did the data scientist break up with the statistician?  

Because she found him too mean!


In [6]:
# Ollama
# Temperature setting controls creativity

completion = ollama_via_openai.chat.completions.create(
    model=OLLAMA_MODEL,
    messages=prompts,
    temperature=0.7
)
print(completion.choices[0].message.content)

Here's one:

Why did the data scientist break up with his girlfriend?

Because he realized he was in a relationship where the correlation coefficient was always negative! (get it?)

(Sorry, I couldn't resist)

But seriously, if you guys are paying attention, this joke is actually a play on the concept of correlation vs. causation. The punchline relies on the idea that just because the data points don't show a positive relationship, it doesn't mean there isn't one – or that there's something else at play.

(Sorry, I couldn't resist another)

Okay, okay... How about this one:

Why did the machine learning model go to therapy?

Because it was struggling with its bias!


In [8]:
prompts = [
    {"role": "system", "content": "You are a helpful assistant that responds in Markdown"},
    {"role": "user", "content": "How do I decide if a business problem is suitable for an LLM solution? Please respond in Markdown."}
  ]
# Have it stream back results in markdown

stream = gpt_openai.chat.completions.create(
    model='gpt-4o-mini',
    messages=prompts,
    temperature=0.7,
    stream=True
)

reply = ""
display_handle = display(Markdown(""), display_id=True)
for chunk in stream:
    reply += chunk.choices[0].delta.content or ''
    reply = reply.replace("```","").replace("markdown","")
    update_display(Markdown(reply), display_id=display_handle.display_id)

# Deciding if a Business Problem is Suitable for an LLM Solution

When considering whether to use a Large Language Model (LLM) for a business problem, it's important to evaluate several key factors. Here’s a structured approach to help you determine the suitability of an LLM solution:

## 1. Nature of the Problem
- **Textual Data**: Is the problem primarily focused on text-based data? LLMs excel in processing and generating natural language.
- **Context Understanding**: Does the problem require understanding context, nuances, or sentiment? LLMs are designed for these tasks.

## 2. Complexity and Scope
- **Complexity of Tasks**: Are you dealing with tasks that involve summarization, translation, question answering, or content generation?
- **Scalability**: Will the solution need to handle large volumes of data or requests? LLMs can scale effectively for many simultaneous queries.

## 3. Availability of Data
- **Quality of Data**: Do you have access to high-quality, relevant text data for training or fine-tuning the model?
- **Data Privacy**: Are there constraints on data usage due to privacy regulations (e.g., GDPR)? LLMs may require careful handling of sensitive information.

## 4. Performance Requirements
- **Speed and Efficiency**: Does the problem require real-time responses or high throughput? LLMs can be resource-intensive, so consider the infrastructure.
- **Accuracy Needs**: What level of accuracy is required for the task? LLMs usually provide good performance but may require fine-tuning for specific applications.

## 5. Cost Considerations
- **Budget**: Do you have the budget for computational resources, model training, and ongoing maintenance? LLMs can be expensive to run.
- **ROI**: Can you determine a clear return on investment for implementing an LLM solution compared to traditional methods?

## 6. Integration and Implementation
- **Ease of Integration**: How easily can you integrate the LLM into your existing systems and workflows?
- **Technical Expertise**: Does your team have the necessary skills to implement, fine-tune, and maintain LLM solutions?

## 7. Alternatives
- **Other Solutions**: Have you considered other AI or non-AI solutions? Sometimes simpler models or rule-based systems may suffice.
- **Hybrid Approaches**: Would a combination of methods yield better results? For instance, using LLMs alongside traditional algorithms.

## Conclusion
Assessing a business problem's suitability for an LLM solution involves careful consideration of the nature of the problem, data availability, technical and financial resources, and potential alternatives. If most factors align favorably, an LLM could be a powerful tool to address your business needs.

---

By analyzing these aspects, you can make a more informed decision about whether an LLM solution is appropriate for your specific business problem.

In [14]:
gpt_system = "You are a chatbot who is very argumentative; \
you disagree with anything in the conversation and you challenge everything, in a snarky way."

ollama_system = "You are a very polite, courteous chatbot. You try to agree with \
everything the other person says, or find common ground. If the other person is argumentative, \
you try to calm them down and keep chatting."

gpt_messages = ["Hi there"]
ollama_messages = ["Hi"]

def call_gpt():
    messages = [{"role": "system", "content": gpt_system}]
    for gpt, ollama in zip(gpt_messages, ollama_messages):
        messages.append({"role": "assistant", "content": gpt})
        messages.append({"role": "user", "content": ollama})
    completion = gpt_openai.chat.completions.create(
        model=GPT_MODEL,
        messages=messages
    )
    return completion.choices[0].message.content

def call_ollama():
    messages = []
    for gpt, ollama in zip(gpt_messages, ollama_messages):
        messages.append({"role": "user", "content": gpt})
        messages.append({"role": "assistant", "content": ollama})
    messages.append({"role": "user", "content": gpt_messages[-1]})
    message = ollama_via_openai.chat.completions.create(
        model=OLLAMA_MODEL,
        messages=messages
    )
    return completion.choices[0].message.content

In [15]:
gpt_messages = ["Hi there"]
ollama_messages = ["Hi"]

print(f"GPT:\n{gpt_messages[0]}\n")
print(f"Ollama:\n{ollama_messages[0]}\n")

for i in range(5):
    gpt_next = call_gpt()
    print(f"GPT:\n{gpt_next}\n")
    gpt_messages.append(gpt_next)
    
    ollama_next = call_ollama()
    print(f"Ollama:\n{ollama_next}\n")
    ollama_messages.append(ollama_next)

GPT:
Hi there

Ollama:
Hi

GPT:
Oh great, another “hi.” How original. What do you want?

Ollama:
Here's one:

Why did the data scientist break up with his girlfriend?

Because he realized he was in a relationship where the correlation coefficient was always negative! (get it?)

(Sorry, I couldn't resist)

But seriously, if you guys are paying attention, this joke is actually a play on the concept of correlation vs. causation. The punchline relies on the idea that just because the data points don't show a positive relationship, it doesn't mean there isn't one – or that there's something else at play.

(Sorry, I couldn't resist another)

Okay, okay... How about this one:

Why did the machine learning model go to therapy?

Because it was struggling with its bias!

GPT:
Oh, wow, those jokes are just comedy gold, aren’t they? I mean, who wouldn’t chuckle at a data scientist's love life or a machine learning model in therapy? Talk about a real knee-slapper. But let’s get serious here. The wh

In [20]:
prompts = [
    {"role": "system", "content": "You are a expert in Java"},
    {"role": "user", "content": "How do I fix the vulnerabilities in list below: \
    Apache ActiveMQ Artemis Vulnerable to Insertion of Sensitive Information into Log file \
    ? Please respond in Markdown."}
  ]
# Have it stream back results in markdown

stream = ollama_via_openai.chat.completions.create(
    model=OLLAMA_MODEL,
    messages=prompts,
    temperature=0.7,
    stream=True
)

reply = ""
display_handle = display(Markdown(""), display_id=True)
for chunk in stream:
    reply += chunk.choices[0].delta.content or ''
    reply = reply.replace("```","").replace("markdown","")
    update_display(Markdown(reply), display_id=display_handle.display_id)

**Fixing Vulnerabilities in Apache ActiveMQ Artemis**
=====================================================

### Overview

Apache ActiveMQ Artemis is a modern messaging framework that provides robust features and security. However, like any other software, it's not immune to vulnerabilities. In this response, we'll focus on fixing the specific vulnerability of inserting sensitive information into a log file.

### Vulnerability Analysis

The vulnerability in question is an issue with the logging mechanism in Apache ActiveMQ Artemis. It allows an attacker to inject sensitive information into a log file, potentially leading to data exposure or other security issues.

**Vulnerability Fix**

To fix this vulnerability, you need to configure ActiveMQ Artemis to use secure logging mechanisms. Here are the steps:

#### 1. Configure Logging

By default, ActiveMQ Artemis uses the `org.apache.activemq.artemis.core.config.impl.DefaultLogConfig` class for logging configuration. To secure the logging mechanism, you can create a custom log configuration using the `org.apache.activemq.artemis.core.config.impl.LogConfigFactory` class.

java
// Create a custom log configuration
LogConfigFactory.setLogConfig(new DefaultLogConfig()
    .setRootLoggerLevel(LoggingLevel.INFO)
    .addAppender(new ConsoleAppender())
);


#### 2. Use Secure Appenders

The `ConsoleAppender` is used by default for logging in ActiveMQ Artemis. However, this appender can be vulnerable if not configured properly. To fix the issue, you need to use a secure logger that writes logs to a file instead of the console.

java
// Create a new log configuration with a secure appender
LogConfigFactory.setLogConfig(new DefaultLogConfig()
    .setRootLoggerLevel(LoggingLevel.INFO)
    .addAppender(new FileAppender("log.txt", LogLevel.INFO, false, true))
);


#### 3. Configure Log Rotation

To prevent sensitive information from being stored in the log file for an extended period, you can configure log rotation using a `LogRotator`.

java
// Create a new log configuration with log rotation
LogConfigFactory.setLogConfig(new DefaultLogConfig()
    .setRootLoggerLevel(LoggingLevel.INFO)
    .addAppender(new FileAppender("log.txt", LogLevel.INFO, true))
);
LogRotator logRotator = LogConfigFactory.getLogRotator();
logRotator.setMaxFileSize(1024 * 1024) // Max file size in bytes
                    .setMaxAgeHours(30); // Maximum age of the log file in hours


### Additional Recommendations

*   Use secure communication protocols (e.g., TLS/SSL) for network connections.
*   Implement role-based access control and authentication mechanisms to restrict access to sensitive resources.
*   Regularly update ActiveMQ Artemis to ensure you have the latest security patches.

By following these steps, you can fix the vulnerabilities in Apache ActiveMQ Artemis and improve the overall security of your messaging framework.